# AgentOps Basic Monitoring 
This is an example of how to use the AgentOps library for basic Agent monitoring with OpenAI's GPT

In [1]:
from openai import OpenAI
import agentops
import os
from dotenv import load_dotenv

Next, we'll grab our two API keys. You can use dotenv like below or however else you like to load environment variables

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or "<your_openai_key>"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your_agentops_key>"

The AgentOps library is designed to be a plug-and-play replacement for the OpenAI Client, maximizing use with minimal install effort.

In [3]:
openai = OpenAI(api_key=OPENAI_API_KEY)
agentops.init(AGENTOPS_API_KEY, tags=["openai-gpt-notebook"])

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=4d085e0e-0b07-409e-a4b4-065d84db1ced


Now just use OpenAI as you would normally!

## Single Session with ChatCompletion

In [4]:
message = ({"role": "user", "content": "Write a 12 word poem about secret agents."},)
res = openai.chat.completions.create(
    model="gpt-3.5-turbo", messages=message, temperature=0.5, stream=True
)

Make sure to end your session with a `Result` (Success|Fail|Indeterminate) for better tracking

In [5]:
agentops.end_session("Success")

🖇 AgentOps: This run's cost $0.00
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=4d085e0e-0b07-409e-a4b4-065d84db1ced


Now if you check the AgentOps dashboard, you should see information related to this run!

# Events
Additionally, you can track custom events via AgentOps.
Let's start a new session and record some events 

In [6]:
# Create new session
agentops.start_session(tags=["openai-gpt-notebook-events"])

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=380a7dc0-2075-4bb6-916f-1cb4553a4379


The easiest way to record actions is through the use of AgentOp's decorators

In [7]:
from agentops import record_function


@record_function("add numbers")
def add(x, y):
    return x + y


add(2, 4)

6

We can also manually craft an event exactly the way we want

In [8]:
from agentops import ActionEvent

message = ({"role": "user", "content": "Hello"},)
response = openai.chat.completions.create(
    model="gpt-3.5-turbo", messages=message, temperature=0.5
)

if "hello" in str(response.choices[0].message.content).lower():
    agentops.record(
        ActionEvent(
            action_type="Agent says hello",
            params=str(message),
            returns=str(response.choices[0].message.content),
        )
    )

In [9]:
agentops.end_session("Success")

🖇 AgentOps: This run's cost $0.000017
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=380a7dc0-2075-4bb6-916f-1cb4553a4379
